In [2]:
#getting the data
!wget https://www.gutenberg.org/files/1661/1661-0.txt -O book.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
text = open("book.txt", 'r', encoding='utf-8').read()
text = text.lower()

In [5]:
sentences = text.split('\n')

In [6]:
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [7]:
token = Tokenizer(oov_token='<UNK>')

In [8]:
token.fit_on_texts(sentences)

In [9]:
vocab_size = len(token.word_index)+1
vocab_size

8915

In [10]:
sequences = token.texts_to_sequences(sentences)
sequences

[[4775, 145, 132, 886, 5, 2, 1093, 5, 130, 35, 46, 656, 2661, 2662],
 [],
 [31, 886, 16, 24, 2, 273, 5, 395, 2233, 9, 2, 657, 447, 3],
 [140, 91, 1676, 5, 2, 693, 22, 52, 1677, 3, 19, 573, 52, 3395],
 [3396, 14, 75, 818, 11, 214, 11, 125, 64, 2234, 273, 11, 263, 2, 465],
 [5, 2, 145, 132, 574, 2235, 19, 31, 886, 64, 2236, 22],
 [1209, 132, 1210, 69, 14, 48, 26, 1468, 9, 2, 657, 447, 14],
 [66, 18, 4, 1469, 2, 1094, 5, 2, 321, 119, 14, 48, 1468, 89],
 [1095, 31, 886],
 [],
 [2663, 2, 1093, 5, 130, 35],
 [],
 [4776, 656, 2661, 2662],
 [],
 [4777, 1020, 4778, 4779, 4780, 886, 4781],
 [140, 2664, 3397, 75, 3398, 4782],
 [],
 [3399, 1096],
 [],
 [575, 244, 4783, 4784, 1470],
 [],
 [2237, 46, 51, 4785, 145, 132, 3400, 3, 4786, 4787],
 [],
 [724, 5, 2, 145, 132, 886, 2, 1093, 5, 130, 35],
 [],
 [1678],
 [],
 [],
 [],
 [],
 [2, 1093, 5, 130, 35],
 [],
 [46, 656, 2661, 2662],
 [],
 [],
 [1927],
 [],
 [7, 6, 887, 9, 943],
 [2665, 2, 249, 483, 694],
 [2666, 6, 114, 5, 2238],
 [3401, 2, 776, 1471,

In [11]:
#Converting into a sequence
input_sequence = []
for sequence in sequences:
  for i in range(1, len(sequence)):
    n_gram_sequence = sequence[:i+1]
    input_sequence.append(n_gram_sequence)

In [12]:
input_sequence[0], input_sequence[1], input_sequence[2]

([4775, 145], [4775, 145, 132], [4775, 145, 132, 886])

In [13]:
#Pad the sequence -> Finding the max len of the sequence

#Max Sequence Length
max_seq_len = max([len(seq) for seq in  input_sequence])
max_seq_len

20

In [14]:
#Padding the sequences
pad_sequence = pad_sequences(sequences, maxlen=max_seq_len)

In [15]:
len(pad_sequence[0]), pad_sequence[0]

(20,
 array([   0,    0,    0,    0,    0,    0, 4775,  145,  132,  886,    5,
           2, 1093,    5,  130,   35,   46,  656, 2661, 2662]))

In [16]:
#Input and Output Pairs
import numpy as np 
padded_sequences = np.array(pad_sequence)

In [17]:
X = padded_sequences[:,:-1]
labels = padded_sequences[:, -1]
Y = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [18]:
X.shape, Y.shape

((12305, 19), (12305, 8915))

In [19]:
#Build and Train Sequences
#LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [20]:
#Defining the Model
model = Sequential() #Sequential model
model.add(Embedding(vocab_size, 100, input_length=max_seq_len-1)) 
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation="softmax"))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 100)           891500    
                                                                 
 bidirectional (Bidirectiona  (None, 512)              731136    
 l)                                                              
                                                                 
 dense (Dense)               (None, 8915)              4573395   
                                                                 
Total params: 6,196,031
Trainable params: 6,196,031
Non-trainable params: 0
_________________________________________________________________


In [21]:
#initializing the callback for early stopping the training if there is no proper improvement in the accuracy
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='acc', min_delta=0.01)

In [22]:
model.fit(X,Y, epochs=50, verbose=1, batch_size=528, callbacks=[es])

Epoch 1/50
24/24 [==============================] - 28s 1s/step - loss: 5.7787 - acc: 0.2170
Epoch 2/50
24/24 [==============================] - 25s 1s/step - loss: 4.3831 - acc: 0.3408


In [ ]:
#Checking the working
seed_text = 'I could not help laughing at the ease with which he explained'
next_words = 100
for i in range(next_words):
  sequence = token.texts_to_sequences([seed_text])
  padded = pad_sequences(sequence, maxlen=max_seq_len-1)
  predicted = model.predict(padded, verbose=0)
  output_word = ''
  for word, index in token.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += ' ' + output_word
seed_text